In [2]:
from pyspark.sql import pandas
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
conf.set("spark.sql.execution.arrow.enabled","true")

spark = SparkSession.builder \
.appName("CENSO") \
.config(conf=conf) \
.getOrCreate()

sqlCtx = SQLContext(spark)

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:736)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:271)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1105)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1091)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:571)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1633)
	at org.apache.spark.SparkContext.$anonfun$new$13(SparkContext.scala:510)
	at org.apache.spark.SparkContext.$anonfun$new$13$adapted(SparkContext.scala:510)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:510)
	at org.apache.spark.api.java.JavaSparkC

In [3]:
class Aws_importer():
    def save_s3_data(self, df,bucket,folder):
        df.write.mode('overwrite').csv('s3a://alex-group2-sptech-bucket-{}/{}'.format(bucket,folder))

    def save_data_client_side(self, df,table):
        df.write \
            .format('com.microsoft.sqlserver.jdbc.spark') \
            .mode('overwrite') \
            .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
            .option('url', 'jdbc:sqlserver://grupo02alexandre.database.windows.net:1433;databaseName=grupo02alexandre;') \
            .option('dbtable', 'dbo.{}'.format(table)) \
            .option('user', 'urubu100') \
            .option('password', 'PASSWORD') \
            .save()
        
class Pandas_Spark():
    #transform pandas dataframe into spark dataframe
    def equivalent_type(self,f):
        if f == 'datetime64[ns]': return TimestampType()
        elif f == 'int64': return LongType()
        elif f == 'int32': return IntegerType()
        elif f == 'float64': return DoubleType()
        elif f == 'float32': return FloatType()
        else: return StringType()

    def define_structure(self,string, format_type):
        try: typo = equivalent_type(format_type)
        except: typo = StringType()
        return StructField(string, typo)

    # Given pandas dataframe, it will return a spark's dataframe.
    def pandas_to_spark(self,pandas_df):
        columns = list(pandas_df.columns)
        types = list(pandas_df.dtypes)
        struct_list = []
        for column, typo in zip(columns, types): 
          struct_list.append(define_structure(column, typo))
        p_schema = StructType(struct_list)
        return sqlCtx.createDataFrame(pandas_df, p_schema)

s3 = Aws_importer()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from pyspark.sql import functions as F
from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql.functions import sum, max, min, avg, count, mean, round, col
import json
import functools

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
customType = StructType([
    StructField('NU_ANO_CENSO',StringType()),
    StructField('NO_REGIAO',StringType()),
    StructField('CO_REGIAO',StringType()),
    StructField('NO_UF',StringType()),
    StructField('SG_UF',StringType()),
    StructField('CO_UF',StringType()),
    StructField('NO_MUNICIPIO',StringType()),
    StructField('CO_MUNICIPIO',StringType()),
    StructField('NO_MESORREGIAO',StringType()),
    StructField('CO_MESORREGIAO',StringType()),
    StructField('NO_MICRORREGIAO',StringType()),
    StructField('CO_MICRORREGIAO',StringType()),
    StructField('CO_DISTRITO',StringType()),
    StructField('CO_ENTIDADE',StringType()),
    StructField('NO_ENTIDADE',StringType()),
    StructField('TP_DEPENDENCIA',StringType()),
    StructField('TP_CATEGORIA_ESCOLA_PRIVADA',StringType()),
    StructField('TP_LOCALIZACAO',StringType()),
    StructField('TP_LOCALIZACAO_DIFERENCIADA',StringType()),
    StructField('DS_ENDERECO',StringType()),
    StructField('NU_ENDERECO',StringType()),
    StructField('DS_COMPLEMENTO',StringType()),
    StructField('NO_BAIRRO',StringType()),
    StructField('CO_CEP',StringType()),
    StructField('NU_DDD',StringType()),
    StructField('NU_TELEFONE',StringType()),
    StructField('TP_SITUACAO_FUNCIONAMENTO',StringType()),
    StructField('CO_ORGAO_REGIONAL',StringType()),
    StructField('DT_ANO_LETIVO_INICIO',StringType()),
    StructField('DT_ANO_LETIVO_TERMINO',StringType()),
    StructField('IN_VINCULO_SECRETARIA_EDUCACAO',StringType()),
    StructField('IN_VINCULO_SEGURANCA_PUBLICA',StringType()),
    StructField('IN_VINCULO_SECRETARIA_SAUDE',StringType()),
    StructField('IN_VINCULO_OUTRO_ORGAO',StringType()),
    StructField('IN_CONVENIADA_PP',StringType()),
    StructField('TP_CONVENIO_PODER_PUBLICO',StringType()),
    StructField('IN_MANT_ESCOLA_PRIVADA_EMP',StringType()),
    StructField('IN_MANT_ESCOLA_PRIVADA_ONG',StringType()),
    StructField('IN_MANT_ESCOLA_PRIVADA_OSCIP',StringType()),
    StructField('IN_MANT_ESCOLA_PRIV_ONG_OSCIP',StringType()),
    StructField('IN_MANT_ESCOLA_PRIVADA_SIND',StringType()),
    StructField('IN_MANT_ESCOLA_PRIVADA_SIST_S',StringType()),
    StructField('IN_MANT_ESCOLA_PRIVADA_S_FINS',StringType()),
    StructField('NU_CNPJ_ESCOLA_PRIVADA',StringType()),
    StructField('NU_CNPJ_MANTENEDORA',StringType()),
    StructField('TP_REGULAMENTACAO',StringType()),
    StructField('TP_RESPONSAVEL_REGULAMENTACAO',StringType()),
    StructField('CO_ESCOLA_SEDE_VINCULADA',StringType()),
    StructField('CO_IES_OFERTANTE',StringType()),
    StructField('IN_LOCAL_FUNC_PREDIO_ESCOLAR',StringType()),
    StructField('TP_OCUPACAO_PREDIO_ESCOLAR',StringType()),
    StructField('IN_LOCAL_FUNC_SALAS_EMPRESA',StringType()),
    StructField('IN_LOCAL_FUNC_SOCIOEDUCATIVO',StringType()),
    StructField('IN_LOCAL_FUNC_UNID_PRISIONAL',StringType()),
    StructField('IN_LOCAL_FUNC_PRISIONAL_SOCIO',StringType()),
    StructField('IN_LOCAL_FUNC_TEMPLO_IGREJA',StringType()),
    StructField('IN_LOCAL_FUNC_CASA_PROFESSOR',StringType()),
    StructField('IN_LOCAL_FUNC_GALPAO',StringType()),
    StructField('TP_OCUPACAO_GALPAO',StringType()),
    StructField('IN_LOCAL_FUNC_SALAS_OUTRA_ESC',StringType()),
    StructField('IN_LOCAL_FUNC_OUTROS',StringType()),
    StructField('IN_PREDIO_COMPARTILHADO',StringType()),
    StructField('IN_AGUA_FILTRADA',IntegerType()),
    StructField('IN_AGUA_POTAVEL',IntegerType()),
    StructField('IN_AGUA_REDE_PUBLICA',IntegerType()),
    StructField('IN_AGUA_POCO_ARTESIANO',IntegerType()),
    StructField('IN_AGUA_CACIMBA',IntegerType()),
    StructField('IN_AGUA_FONTE_RIO',IntegerType()),
    StructField('IN_AGUA_INEXISTENTE',IntegerType()),
    StructField('IN_ENERGIA_REDE_PUBLICA',IntegerType()),
    StructField('IN_ENERGIA_GERADOR',IntegerType()),
    StructField('IN_ENERGIA_GERADOR_FOSSIL',IntegerType()),
    StructField('IN_ENERGIA_OUTROS',IntegerType()),
    StructField('IN_ENERGIA_RENOVAVEL',IntegerType()),
    StructField('IN_ENERGIA_INEXISTENTE',IntegerType()),
    StructField('IN_ESGOTO_REDE_PUBLICA',IntegerType()),
    StructField('IN_ESGOTO_FOSSA_SEPTICA',IntegerType()),
    StructField('IN_ESGOTO_FOSSA_COMUM',IntegerType()),
    StructField('IN_ESGOTO_FOSSA',IntegerType()),
    StructField('IN_ESGOTO_INEXISTENTE',IntegerType()),
    StructField('IN_LIXO_SERVICO_COLETA',IntegerType()),
    StructField('IN_LIXO_QUEIMA',IntegerType()),
    StructField('IN_LIXO_ENTERRA',IntegerType()),
    StructField('IN_LIXO_DESTINO_FINAL_PUBLICO',IntegerType()),
    StructField('IN_LIXO_DESCARTA_OUTRA_AREA',IntegerType()),
    StructField('IN_LIXO_JOGA_OUTRA_AREA',IntegerType()),
    StructField('IN_LIXO_OUTROS',IntegerType()),
    StructField('IN_LIXO_RECICLA',IntegerType()),
    StructField('IN_TRATAMENTO_LIXO_SEPARACAO',IntegerType()),
    StructField('IN_TRATAMENTO_LIXO_REUTILIZA',IntegerType()),
    StructField('IN_TRATAMENTO_LIXO_RECICLAGEM',IntegerType()),
    StructField('IN_TRATAMENTO_LIXO_INEXISTENTE',IntegerType()),
    StructField('IN_ALMOXARIFADO',IntegerType()),
    StructField('IN_AREA_VERDE',IntegerType()),
    StructField('IN_AUDITORIO',IntegerType()),
    StructField('IN_BANHEIRO_FORA_PREDIO',IntegerType()),
    StructField('IN_BANHEIRO_DENTRO_PREDIO',IntegerType()),
    StructField('IN_BANHEIRO',IntegerType()),
    StructField('IN_BANHEIRO_EI',IntegerType()),
    StructField('IN_BANHEIRO_PNE',IntegerType()),
    StructField('IN_BANHEIRO_FUNCIONARIOS',IntegerType()),
    StructField('IN_BANHEIRO_CHUVEIRO',IntegerType()),
    StructField('IN_BERCARIO',IntegerType()),
    StructField('IN_BIBLIOTECA',IntegerType()),
    StructField('IN_BIBLIOTECA_SALA_LEITURA',IntegerType()),
    StructField('IN_COZINHA',IntegerType()),
    StructField('IN_DESPENSA',IntegerType()),
    StructField('IN_DORMITORIO_ALUNO',IntegerType()),
    StructField('IN_DORMITORIO_PROFESSOR',IntegerType()),
    StructField('IN_LABORATORIO_CIENCIAS',IntegerType()),
    StructField('IN_LABORATORIO_INFORMATICA',IntegerType()),
    StructField('IN_PATIO_COBERTO',IntegerType()),
    StructField('IN_PATIO_DESCOBERTO',IntegerType()),
    StructField('IN_PARQUE_INFANTIL',IntegerType()),
    StructField('IN_PISCINA',IntegerType()),
    StructField('IN_QUADRA_ESPORTES',IntegerType()),
    StructField('IN_QUADRA_ESPORTES_COBERTA',IntegerType()),
    StructField('IN_QUADRA_ESPORTES_DESCOBERTA',IntegerType()),
    StructField('IN_REFEITORIO',IntegerType()),
    StructField('IN_SALA_ATELIE_ARTES',IntegerType()),
    StructField('IN_SALA_MUSICA_CORAL',IntegerType()),
    StructField('IN_SALA_ESTUDIO_DANCA',IntegerType()),
    StructField('IN_SALA_MULTIUSO',IntegerType()),
    StructField('IN_SALA_DIRETORIA',IntegerType()),
    StructField('IN_SALA_LEITURA',IntegerType()),
    StructField('IN_SALA_PROFESSOR',IntegerType()),
    StructField('IN_SALA_REPOUSO_ALUNO',IntegerType()),
    StructField('IN_SECRETARIA',IntegerType()),
    StructField('IN_SALA_ATENDIMENTO_ESPECIAL',IntegerType()),
    StructField('IN_TERREIRAO',IntegerType()),
    StructField('IN_VIVEIRO',IntegerType()),
    StructField('IN_DEPENDENCIAS_PNE',IntegerType()),
    StructField('IN_LAVANDERIA',IntegerType()),
    StructField('IN_DEPENDENCIAS_OUTRAS',IntegerType()),
    StructField('IN_ACESSIBILIDADE_CORRIMAO',IntegerType()),
    StructField('IN_ACESSIBILIDADE_ELEVADOR',IntegerType()),
    StructField('IN_ACESSIBILIDADE_PISOS_TATEIS',IntegerType()),
    StructField('IN_ACESSIBILIDADE_VAO_LIVRE',IntegerType()),
    StructField('IN_ACESSIBILIDADE_RAMPAS',IntegerType()),
    StructField('IN_ACESSIBILIDADE_SINAL_SONORO',IntegerType()),
    StructField('IN_ACESSIBILIDADE_SINAL_TATIL',IntegerType()),
    StructField('IN_ACESSIBILIDADE_SINAL_VISUAL',IntegerType()),
    StructField('IN_ACESSIBILIDADE_INEXISTENTE',IntegerType()),
    StructField('QT_SALAS_EXISTENTES',IntegerType()),
    StructField('QT_SALAS_UTILIZADAS_DENTRO',IntegerType()),
    StructField('QT_SALAS_UTILIZADAS_FORA',IntegerType()),
    StructField('QT_SALAS_UTILIZADAS',IntegerType()),
    StructField('QT_SALAS_UTILIZA_CLIMATIZADAS',IntegerType()),
    StructField('QT_SALAS_UTILIZADAS_ACESSIVEIS',IntegerType()),
    StructField('IN_EQUIP_PARABOLICA',IntegerType()),
    StructField('IN_COMPUTADOR',IntegerType()),
    StructField('IN_EQUIP_COPIADORA',IntegerType()),
    StructField('IN_EQUIP_IMPRESSORA',IntegerType()),
    StructField('IN_EQUIP_IMPRESSORA_MULT',IntegerType()),
    StructField('IN_EQUIP_SCANNER',IntegerType()),
    StructField('IN_EQUIP_NENHUM',IntegerType()),
    StructField('IN_EQUIP_DVD',IntegerType()),
    StructField('QT_EQUIP_DVD',IntegerType()),
    StructField('IN_EQUIP_SOM',IntegerType()),
    StructField('QT_EQUIP_SOM',IntegerType()),
    StructField('IN_EQUIP_TV',IntegerType()),
    StructField('QT_EQUIP_TV',IntegerType()),
    StructField('IN_EQUIP_LOUSA_DIGITAL',IntegerType()),
    StructField('QT_EQUIP_LOUSA_DIGITAL',IntegerType()),
    StructField('IN_EQUIP_MULTIMIDIA',IntegerType()),
    StructField('QT_EQUIP_MULTIMIDIA',IntegerType()),
    StructField('IN_EQUIP_VIDEOCASSETE',IntegerType()),
    StructField('IN_EQUIP_RETROPROJETOR',IntegerType()),
    StructField('IN_EQUIP_FAX',IntegerType()),
    StructField('IN_EQUIP_FOTO',IntegerType()),
    StructField('QT_EQUIP_VIDEOCASSETE',IntegerType()),
    StructField('QT_EQUIP_PARABOLICA',IntegerType()),
    StructField('QT_EQUIP_COPIADORA',IntegerType()),
    StructField('QT_EQUIP_RETROPROJETOR',IntegerType()),
    StructField('QT_EQUIP_IMPRESSORA',IntegerType()),
    StructField('QT_EQUIP_IMPRESSORA_MULT',IntegerType()),
    StructField('QT_EQUIP_FAX',IntegerType()),
    StructField('QT_EQUIP_FOTO',IntegerType()),
    StructField('QT_COMP_ALUNO',IntegerType()),
    StructField('IN_DESKTOP_ALUNO',IntegerType()),
    StructField('QT_DESKTOP_ALUNO',IntegerType()),
    StructField('IN_COMP_PORTATIL_ALUNO',IntegerType()),
    StructField('QT_COMP_PORTATIL_ALUNO',IntegerType()),
    StructField('IN_TABLET_ALUNO',IntegerType()),
    StructField('QT_TABLET_ALUNO',IntegerType()),
    StructField('QT_COMPUTADOR',IntegerType()),
    StructField('QT_COMP_ADMINISTRATIVO',IntegerType()),
    StructField('IN_INTERNET',IntegerType()),
    StructField('IN_INTERNET_ALUNOS',IntegerType()),
    StructField('IN_INTERNET_ADMINISTRATIVO',IntegerType()),
    StructField('IN_INTERNET_APRENDIZAGEM',IntegerType()),
    StructField('IN_INTERNET_COMUNIDADE',IntegerType()),
    StructField('IN_ACESSO_INTERNET_COMPUTADOR',IntegerType()),
    StructField('IN_ACES_INTERNET_DISP_PESSOAIS',IntegerType()),
    StructField('TP_REDE_LOCAL',IntegerType()),
    StructField('IN_BANDA_LARGA',IntegerType()),
    StructField('QT_FUNCIONARIOS',IntegerType()),
    StructField('IN_PROF_ADMINISTRATIVOS',IntegerType()),
    StructField('QT_PROF_ADMINISTRATIVOS',IntegerType()),
    StructField('IN_PROF_SERVICOS_GERAIS',IntegerType()),
    StructField('QT_PROF_SERVICOS_GERAIS',IntegerType()),
    StructField('IN_PROF_BIBLIOTECARIO',IntegerType()),
    StructField('QT_PROF_BIBLIOTECARIO',IntegerType()),
    StructField('IN_PROF_SAUDE',IntegerType()),
    StructField('QT_PROF_SAUDE',IntegerType()),
    StructField('IN_PROF_COORDENADOR',IntegerType()),
    StructField('QT_PROF_COORDENADOR',IntegerType()),
    StructField('IN_PROF_FONAUDIOLOGO',IntegerType()),
    StructField('QT_PROF_FONAUDIOLOGO',IntegerType()),
    StructField('IN_PROF_NUTRICIONISTA',IntegerType()),
    StructField('QT_PROF_NUTRICIONISTA',IntegerType()),
    StructField('IN_PROF_PSICOLOGO',IntegerType()),
    StructField('QT_PROF_PSICOLOGO',IntegerType()),
    StructField('IN_PROF_ALIMENTACAO',IntegerType()),
    StructField('QT_PROF_ALIMENTACAO',IntegerType()),
    StructField('IN_PROF_PEDAGOGIA',IntegerType()),
    StructField('QT_PROF_PEDAGOGIA',IntegerType()),
    StructField('IN_PROF_SECRETARIO',IntegerType()),
    StructField('QT_PROF_SECRETARIO',IntegerType()),
    StructField('IN_PROF_SEGURANCA',IntegerType()),
    StructField('QT_PROF_SEGURANCA',IntegerType()),
    StructField('IN_PROF_MONITORES',IntegerType()),
    StructField('QT_PROF_MONITORES',IntegerType()),
    StructField('IN_PROF_GESTAO',IntegerType()),
    StructField('QT_PROF_GESTAO',IntegerType()),
    StructField('IN_PROF_ASSIST_SOCIAL',IntegerType()),
    StructField('QT_PROF_ASSIST_SOCIAL',IntegerType()),
    StructField('IN_ALIMENTACAO',IntegerType()),
    StructField('IN_SERIE_ANO',IntegerType()),
    StructField('IN_PERIODOS_SEMESTRAIS',IntegerType()),
    StructField('IN_FUNDAMENTAL_CICLOS',IntegerType()),
    StructField('IN_GRUPOS_NAO_SERIADOS',IntegerType()),
    StructField('IN_MODULOS',IntegerType()),
    StructField('IN_FORMACAO_ALTERNANCIA',IntegerType()),
    StructField('IN_MATERIAL_PED_MULTIMIDIA',IntegerType()),
    StructField('IN_MATERIAL_PED_INFANTIL',IntegerType()),
    StructField('IN_MATERIAL_PED_CIENTIFICO',IntegerType()),
    StructField('IN_MATERIAL_PED_DIFUSAO',IntegerType()),
    StructField('IN_MATERIAL_PED_MUSICAL',IntegerType()),
    StructField('IN_MATERIAL_PED_JOGOS',IntegerType()),
    StructField('IN_MATERIAL_PED_ARTISTICAS',IntegerType()),
    StructField('IN_MATERIAL_PED_DESPORTIVA',IntegerType()),
    StructField('IN_MATERIAL_PED_INDIGENA',IntegerType()),
    StructField('IN_MATERIAL_PED_ETNICO',IntegerType()),
    StructField('IN_MATERIAL_PED_CAMPO',IntegerType()),
    StructField('IN_MATERIAL_PED_NENHUM',IntegerType()),
    StructField('IN_MATERIAL_ESP_QUILOMBOLA',IntegerType()),
    StructField('IN_MATERIAL_ESP_INDIGENA',IntegerType()),
    StructField('IN_MATERIAL_ESP_NAO_UTILIZA',IntegerType()),
    StructField('IN_EDUCACAO_INDIGENA',IntegerType()),
    StructField('TP_INDIGENA_LINGUA',IntegerType()),
    StructField('CO_LINGUA_INDIGENA_1',IntegerType()),
    StructField('CO_LINGUA_INDIGENA_2',IntegerType()),
    StructField('CO_LINGUA_INDIGENA_3',IntegerType()),
    StructField('IN_BRASIL_ALFABETIZADO',IntegerType()),
    StructField('IN_FINAL_SEMANA',IntegerType()),
    StructField('IN_EXAME_SELECAO',IntegerType()),
    StructField('IN_RESERVA_PPI',IntegerType()),
    StructField('IN_RESERVA_RENDA',IntegerType()),
    StructField('IN_RESERVA_PUBLICA',IntegerType()),
    StructField('IN_RESERVA_PCD',IntegerType()),
    StructField('IN_RESERVA_OUTROS',IntegerType()),
    StructField('IN_RESERVA_NENHUMA',IntegerType()),
    StructField('IN_REDES_SOCIAIS',IntegerType()),
    StructField('IN_ESPACO_ATIVIDADE',IntegerType()),
    StructField('IN_ESPACO_EQUIPAMENTO',IntegerType()),
    StructField('IN_ORGAO_ASS_PAIS',IntegerType()),
    StructField('IN_ORGAO_ASS_PAIS_MESTRES',IntegerType()),
    StructField('IN_ORGAO_CONSELHO_ESCOLAR',IntegerType()),
    StructField('IN_ORGAO_GREMIO_ESTUDANTIL',IntegerType()),
    StructField('IN_ORGAO_OUTROS',IntegerType()),
    StructField('IN_ORGAO_NENHUM',IntegerType()),
    StructField('TP_PROPOSTA_PEDAGOGICA',IntegerType()),
    StructField('TP_AEE',IntegerType()),
    StructField('TP_ATIVIDADE_COMPLEMENTAR',IntegerType()),
    StructField('IN_MEDIACAO_PRESENCIAL',IntegerType()),
    StructField('IN_MEDIACAO_SEMIPRESENCIAL',IntegerType()),
    StructField('IN_MEDIACAO_EAD',IntegerType()),
    StructField('IN_REGULAR',IntegerType()),
    StructField('IN_DIURNO',IntegerType()),
    StructField('IN_NOTURNO',IntegerType()),
    StructField('IN_EAD',IntegerType()),
    StructField('IN_BAS',IntegerType()),
    StructField('IN_INF',IntegerType()),
    StructField('IN_INF_CRE',IntegerType()),
    StructField('IN_INF_PRE',IntegerType()),
    StructField('IN_FUND',IntegerType()),
    StructField('IN_FUND_AI',IntegerType()),
    StructField('IN_FUND_AF',IntegerType()),
    StructField('IN_MED',IntegerType()),
    StructField('IN_PROF',IntegerType()),
    StructField('IN_PROF_TEC',IntegerType()),
    StructField('IN_EJA',IntegerType()),
    StructField('IN_EJA_FUND',IntegerType()),
    StructField('IN_EJA_MED',IntegerType()),
    StructField('IN_ESP',IntegerType()),
    StructField('IN_ESP_CC',IntegerType()),
    StructField('IN_ESP_CE',IntegerType()),
    StructField('QT_MAT_BAS',IntegerType()),
    StructField('QT_MAT_INF',IntegerType()),
    StructField('QT_MAT_INF_CRE',IntegerType()),
    StructField('QT_MAT_INF_PRE',IntegerType()),
    StructField('QT_MAT_FUND',IntegerType()),
    StructField('QT_MAT_FUND_AI',IntegerType()),
    StructField('QT_MAT_FUND_AF',IntegerType()),
    StructField('QT_MAT_MED',IntegerType()),
    StructField('QT_MAT_PROF',IntegerType()),
    StructField('QT_MAT_PROF_TEC',IntegerType()),
    StructField('QT_MAT_EJA',IntegerType()),
    StructField('QT_MAT_EJA_FUND',IntegerType()),
    StructField('QT_MAT_EJA_MED',IntegerType()),
    StructField('QT_MAT_ESP',IntegerType()),
    StructField('QT_MAT_ESP_CC',IntegerType()),
    StructField('QT_MAT_ESP_CE',IntegerType()),
    StructField('QT_MAT_BAS_FEM',IntegerType()),
    StructField('QT_MAT_BAS_MASC',IntegerType()),
    StructField('QT_MAT_BAS_ND',IntegerType()),
    StructField('QT_MAT_BAS_BRANCA',IntegerType()),
    StructField('QT_MAT_BAS_PRETA',IntegerType()),
    StructField('QT_MAT_BAS_PARDA',IntegerType()),
    StructField('QT_MAT_BAS_AMARELA',IntegerType()),
    StructField('QT_MAT_BAS_INDIGENA',IntegerType()),
    StructField('QT_MAT_BAS_0_3',IntegerType()),
    StructField('QT_MAT_BAS_4_5',IntegerType()),
    StructField('QT_MAT_BAS_6_10',IntegerType()),
    StructField('QT_MAT_BAS_11_14',IntegerType()),
    StructField('QT_MAT_BAS_15_17',IntegerType()),
    StructField('QT_MAT_BAS_18_MAIS',IntegerType()),
    StructField('QT_MAT_BAS_D',IntegerType()),
    StructField('QT_MAT_BAS_N',IntegerType()),
    StructField('QT_MAT_BAS_EAD',IntegerType()),
    StructField('QT_MAT_INF_INT',IntegerType()),
    StructField('QT_MAT_INF_CRE_INT',IntegerType()),
    StructField('QT_MAT_INF_PRE_INT',IntegerType()),
    StructField('QT_MAT_FUND_INT',IntegerType()),
    StructField('QT_MAT_FUND_AI_INT',IntegerType()),
    StructField('QT_MAT_FUND_AF_INT',IntegerType()),
    StructField('QT_MAT_MED_INT',IntegerType()),
    StructField('QT_DOC_BAS',IntegerType()),
    StructField('QT_DOC_INF',IntegerType()),
    StructField('QT_DOC_INF_CRE',IntegerType()),
    StructField('QT_DOC_INF_PRE',IntegerType()),
    StructField('QT_DOC_FUND',IntegerType()),
    StructField('QT_DOC_FUND_AI',IntegerType()),
    StructField('QT_DOC_FUND_AF',IntegerType()),
    StructField('QT_DOC_MED',IntegerType()),
    StructField('QT_DOC_PROF',IntegerType()),
    StructField('QT_DOC_PROF_TEC',IntegerType()),
    StructField('QT_DOC_EJA',IntegerType()),
    StructField('QT_DOC_EJA_FUND',IntegerType()),
    StructField('QT_DOC_EJA_MED',IntegerType()),
    StructField('QT_DOC_ESP',IntegerType()),
    StructField('QT_DOC_ESP_CC',IntegerType()),
    StructField('QT_DOC_ESP_CE',IntegerType()),
    StructField('QT_TUR_BAS',IntegerType()),
    StructField('QT_TUR_INF',IntegerType()),
    StructField('QT_TUR_INF_CRE',IntegerType()),
    StructField('QT_TUR_INF_PRE',IntegerType()),
    StructField('QT_TUR_FUND',IntegerType()),
    StructField('QT_TUR_FUND_AI',IntegerType()),
    StructField('QT_TUR_FUND_AF',IntegerType()),
    StructField('QT_TUR_MED',IntegerType()),
    StructField('QT_TUR_PROF',IntegerType()),
    StructField('QT_TUR_PROF_TEC',IntegerType()),
    StructField('QT_TUR_EJA',IntegerType()),
    StructField('QT_TUR_EJA_FUND',IntegerType()),
    StructField('QT_TUR_EJA_MED',IntegerType()),
    StructField('QT_TUR_ESP',IntegerType()),
    StructField('QT_TUR_ESP_CC',IntegerType()),
    StructField('QT_TUR_ESP_CE',IntegerType())
])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
censo2018DF = spark.read.option('delimiter', ';').option('header', 'true').csv('s3a://alex-group2-sptech-bucket-brutos/microdados_ed_basica_2018.csv',schema=customType)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
censo2019DF = spark.read.option('delimiter', ';').option('header', 'true').csv('s3a://alex-group2-sptech-bucket-brutos/microdados_ed_basica_2019.csv',schema=customType)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
censo2020DF = spark.read.option('delimiter', ';').option('header', 'true').csv('s3a://alex-group2-sptech-bucket-brutos/microdados_ed_basica_2020.CSV',schema=customType)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
censo2021DF = spark.read.option('delimiter', ';').option('header', 'true').csv('s3a://alex-group2-sptech-bucket-brutos/microdados_ed_basica_2021.csv',schema=customType)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
censoEscolarDF = unionAll([censo2018DF,censo2019DF,censo2020DF,censo2021DF])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
censoEscolarDF.select('NO_UF').distinct().show(n=30)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|              NO_UF|
+-------------------+
|          S�o Paulo|
|     Rio de Janeiro|
|           Rond�nia|
|            Alagoas|
|              Bahia|
|              Piau�|
|            Sergipe|
|             Paran�|
|          Tocantins|
|              Cear�|
|     Santa Catarina|
|     Esp�rito Santo|
| Mato Grosso do Sul|
|         Pernambuco|
|  Rio Grande do Sul|
|           Maranh�o|
|       Minas Gerais|
|        Mato Grosso|
|               Par�|
|            Roraima|
|   Distrito Federal|
|              Amap�|
|               Acre|
|            Para�ba|
|              Goi�s|
|           Amazonas|
|Rio Grande do Norte|
+-------------------+

In [15]:
censoEscolarDF = censoEscolarDF.replace('Cear�','Ceará','NO_UF')\
                .replace('Par�','Pará','NO_UF')\
                .replace('Piau�','Piauí','NO_UF')\
                .replace('Amap�','Amapá','NO_UF')\
                .replace('Maranh�o','Maranhão','NO_UF')\
                .replace('Para�ba','Paraíba','NO_UF')\
                .replace('Rond�nia','Rondônia','NO_UF')\
                .replace('Goi�s','Goiás','NO_UF')\
                .replace('Esp�rito Santo','Espírito Santo','NO_UF')\
                .replace('S�o Paulo','São Paulo','NO_UF')\
                .replace('Paran�','Paraná','NO_UF')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
censoEscolarDF.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

910350

In [17]:
len(censoEscolarDF.columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

370

In [18]:
censoEscolarDF = censoEscolarDF.fillna(0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
censoEscolarDF.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

KeyboardInterrupt: 

In [21]:
censoEscolarDF = censoEscolarDF.drop('IN_FINAL_SEMANA','IN_BRASIL_ALFABETIZADO','TP_OCUPACAO_GALPAO','TP_OCUPACAO_PREDIO_ESCOLAR','TP_RESPONSAVEL_REGULAMENTACAO','IN_MANT_ESCOLA_PRIVADA_EMP','SG_UF','CO_CEP','DT_ANO_LETIVO_INICIO','DT_ANO_LETIVO_TERMINO','TP_CONVENIO_PODER_PUBLICO','CO_UF','CO_REGIAO','CO_MUNICIPIO','CO_MESORREGIAO','CO_MICRORREGIAO','CO_DISTRITO','CO_ENTIDADE','NO_ENTIDADE','TP_DEPENDENCIA','TP_CATEGORIA_ESCOLA_PRIVADA','TP_LOCALIZACAO','TP_LOCALIZACAO_DIFERENCIADA','DS_ENDERECO','NU_ENDERECO','DS_COMPLEMENTO','NO_BAIRRO','NU_DDD','NU_TELEFONE','TP_SITUACAO_FUNCIONAMENTO','CO_ORGAO_REGIONAL','QT_TUR_BAS','QT_TUR_INF','QT_TUR_INF_CRE','QT_TUR_INF_PRE','QT_TUR_FUND','QT_TUR_FUND_AI','QT_TUR_FUND_AF','QT_TUR_MED','QT_TUR_PROF','QT_TUR_PROF_TEC','QT_TUR_EJA','QT_TUR_EJA_FUND','QT_TUR_EJA_MED','QT_TUR_ESP','QT_TUR_ESP_CC','QT_TUR_ESP_CE','QT_DOC_BAS','QT_DOC_INF','QT_DOC_INF_CRE','QT_DOC_INF_PRE','QT_DOC_FUND','QT_DOC_FUND_AI','QT_DOC_FUND_AF','QT_DOC_MED','QT_DOC_PROF','QT_DOC_PROF_TEC','QT_DOC_EJA','QT_DOC_EJA_FUND','QT_DOC_EJA_MED','QT_DOC_ESP','QT_DOC_ESP_CC','QT_DOC_ESP_CE','IN_VINCULO_SECRETARIA_EDUCACAO','IN_VINCULO_SEGURANCA_PUBLICA','IN_VINCULO_SECRETARIA_SAUDE','IN_VINCULO_OUTRO_ORGAO','IN_CONVENIADA_PP','IN_MANT_ESCOLA_PRIVADA_ONG','IN_MANT_ESCOLA_PRIVADA_OSCIP','IN_MANT_ESCOLA_PRIV_ONG_OSCIP','IN_MANT_ESCOLA_PRIVADA_SIND','IN_MANT_ESCOLA_PRIVADA_SIST_S','IN_MANT_ESCOLA_PRIVADA_S_FINS','NU_CNPJ_ESCOLA_PRIVADA','NU_CNPJ_MANTENEDORA','TP_REGULAMENTACAO','CO_ESCOLA_SEDE_VINCULADA','CO_IES_OFERTANTE','IN_LOCAL_FUNC_PREDIO_ESCOLAR','IN_LOCAL_FUNC_SALAS_EMPRESA','IN_LOCAL_FUNC_SOCIOEDUCATIVO','IN_LOCAL_FUNC_UNID_PRISIONAL','IN_LOCAL_FUNC_PRISIONAL_SOCIO','IN_LOCAL_FUNC_TEMPLO_IGREJA','IN_LOCAL_FUNC_CASA_PROFESSOR','IN_LOCAL_FUNC_GALPAO','IN_LOCAL_FUNC_SALAS_OUTRA_ESC','IN_LOCAL_FUNC_OUTROS','IN_PREDIO_COMPARTILHADO','IN_TRATAMENTO_LIXO_SEPARACAO','IN_TRATAMENTO_LIXO_REUTILIZA','IN_TRATAMENTO_LIXO_RECICLAGEM','IN_TRATAMENTO_LIXO_INEXISTENTE','IN_ALMOXARIFADO','IN_AREA_VERDE','IN_AUDITORIO','IN_BANHEIRO_FORA_PREDIO','IN_BANHEIRO_DENTRO_PREDIO','IN_BANHEIRO','IN_BANHEIRO_EI','IN_BANHEIRO_PNE','IN_BANHEIRO_FUNCIONARIOS','IN_BANHEIRO_CHUVEIRO','IN_BERCARIO','IN_SALA_ATELIE_ARTES','IN_SALA_MUSICA_CORAL','IN_SALA_ESTUDIO_DANCA','IN_SALA_MULTIUSO','IN_SALA_DIRETORIA','IN_SALA_LEITURA','IN_SALA_PROFESSOR','IN_SALA_REPOUSO_ALUNO','IN_SECRETARIA','IN_SALA_ATENDIMENTO_ESPECIAL','IN_TERREIRAO','IN_VIVEIRO','IN_DEPENDENCIAS_PNE','IN_LAVANDERIA','IN_DEPENDENCIAS_OUTRAS','IN_ACESSIBILIDADE_CORRIMAO','IN_ACESSIBILIDADE_ELEVADOR','IN_ACESSIBILIDADE_PISOS_TATEIS','IN_ACESSIBILIDADE_VAO_LIVRE','IN_ACESSIBILIDADE_RAMPAS','IN_ACESSIBILIDADE_SINAL_SONORO','IN_ACESSIBILIDADE_SINAL_TATIL','IN_ACESSIBILIDADE_SINAL_VISUAL','IN_ACESSIBILIDADE_INEXISTENTE','IN_EQUIP_PARABOLICA','IN_COMPUTADOR','IN_EQUIP_COPIADORA','IN_EQUIP_IMPRESSORA','IN_EQUIP_IMPRESSORA_MULT','IN_EQUIP_SCANNER','IN_EQUIP_NENHUM','IN_EQUIP_DVD','QT_EQUIP_DVD','IN_EQUIP_SOM','QT_EQUIP_SOM','IN_EQUIP_TV','QT_EQUIP_TV','IN_EQUIP_LOUSA_DIGITAL','QT_EQUIP_LOUSA_DIGITAL','IN_EQUIP_MULTIMIDIA','QT_EQUIP_MULTIMIDIA','IN_EQUIP_VIDEOCASSETE','IN_EQUIP_RETROPROJETOR','IN_EQUIP_FAX','IN_EQUIP_FOTO','QT_EQUIP_VIDEOCASSETE','QT_EQUIP_PARABOLICA','QT_EQUIP_COPIADORA','QT_EQUIP_RETROPROJETOR','QT_EQUIP_IMPRESSORA','QT_EQUIP_IMPRESSORA_MULT','QT_EQUIP_FAX','QT_EQUIP_FOTO','QT_COMP_ALUNO','IN_SERIE_ANO','IN_PERIODOS_SEMESTRAIS','IN_FUNDAMENTAL_CICLOS','IN_GRUPOS_NAO_SERIADOS','IN_MODULOS','IN_FORMACAO_ALTERNANCIA','IN_MATERIAL_PED_MULTIMIDIA','IN_MATERIAL_PED_INFANTIL','IN_MATERIAL_PED_CIENTIFICO','IN_MATERIAL_PED_DIFUSAO','IN_MATERIAL_PED_MUSICAL','IN_MATERIAL_PED_JOGOS','IN_MATERIAL_PED_ARTISTICAS','IN_MATERIAL_PED_DESPORTIVA','IN_MATERIAL_PED_INDIGENA','IN_MATERIAL_PED_ETNICO','IN_MATERIAL_PED_CAMPO','IN_MATERIAL_PED_NENHUM','IN_MATERIAL_ESP_QUILOMBOLA','IN_MATERIAL_ESP_INDIGENA','IN_MATERIAL_ESP_NAO_UTILIZA','IN_RESERVA_PPI','IN_RESERVA_RENDA','IN_RESERVA_PUBLICA','IN_RESERVA_PCD','IN_RESERVA_OUTROS','IN_RESERVA_NENHUMA','IN_REDES_SOCIAIS','IN_ESPACO_ATIVIDADE','IN_ESPACO_EQUIPAMENTO','IN_ORGAO_ASS_PAIS','IN_ORGAO_ASS_PAIS_MESTRES','IN_ORGAO_CONSELHO_ESCOLAR','IN_ORGAO_GREMIO_ESTUDANTIL','IN_ORGAO_OUTROS','IN_ORGAO_NENHUM','TP_PROPOSTA_PEDAGOGICA','TP_AEE','TP_ATIVIDADE_COMPLEMENTAR','IN_MEDIACAO_PRESENCIAL','IN_MEDIACAO_SEMIPRESENCIAL','IN_MEDIACAO_EAD','IN_REGULAR','IN_EDUCACAO_INDIGENA','TP_INDIGENA_LINGUA','CO_LINGUA_INDIGENA_1','CO_LINGUA_INDIGENA_2','CO_LINGUA_INDIGENA_3','NO_MUNICIPIO','NO_MESORREGIAO','NO_MICRORREGIAO')

An error was encountered:
Session 2 did not reach idle status in time. Current status is busy.


In [22]:
len(censoEscolarDF.columns)

An error was encountered:
Session 2 did not reach idle status in time. Current status is busy.


In [19]:
censoEscolarDF.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- NU_ANO_CENSO: string (nullable = true)
 |-- NO_REGIAO: string (nullable = true)
 |-- NO_UF: string (nullable = true)
 |-- IN_AGUA_FILTRADA: integer (nullable = true)
 |-- IN_AGUA_POTAVEL: integer (nullable = true)
 |-- IN_AGUA_REDE_PUBLICA: integer (nullable = true)
 |-- IN_AGUA_POCO_ARTESIANO: integer (nullable = true)
 |-- IN_AGUA_CACIMBA: integer (nullable = true)
 |-- IN_AGUA_FONTE_RIO: integer (nullable = true)
 |-- IN_AGUA_INEXISTENTE: integer (nullable = true)
 |-- IN_ENERGIA_REDE_PUBLICA: integer (nullable = true)
 |-- IN_ENERGIA_GERADOR: integer (nullable = true)
 |-- IN_ENERGIA_GERADOR_FOSSIL: integer (nullable = true)
 |-- IN_ENERGIA_OUTROS: integer (nullable = true)
 |-- IN_ENERGIA_RENOVAVEL: integer (nullable = true)
 |-- IN_ENERGIA_INEXISTENTE: integer (nullable = true)
 |-- IN_ESGOTO_REDE_PUBLICA: integer (nullable = true)
 |-- IN_ESGOTO_FOSSA_SEPTICA: integer (nullable = true)
 |-- IN_ESGOTO_FOSSA_COMUM: integer (nullable = true)
 |-- IN_ESGOTO_FOSSA: integer 

In [20]:
s3.save_s3_data(censoEscolarDF,'preprocessado','CENSO_ESCOLAR')
s3.save_s3_data(censoEscolarDF,'cliente','CENSO_ESCOLAR')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Censo Escolar Saneamento**

In [21]:
censoEscolarSaneamentoDF = censoEscolarDF.groupBy('NU_ANO_CENSO','NO_REGIAO','NO_UF').sum('IN_AGUA_FILTRADA','IN_AGUA_POTAVEL','IN_AGUA_REDE_PUBLICA','IN_AGUA_POCO_ARTESIANO','IN_AGUA_CACIMBA','IN_AGUA_FONTE_RIO','IN_AGUA_INEXISTENTE','IN_ESGOTO_REDE_PUBLICA','IN_ESGOTO_FOSSA_SEPTICA','IN_ESGOTO_FOSSA_COMUM','IN_ESGOTO_FOSSA','IN_ESGOTO_INEXISTENTE')

censoEscolarSaneamentoDF.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+-----------------+---------------------+--------------------+-------------------------+---------------------------+--------------------+----------------------+------------------------+---------------------------+----------------------------+--------------------------+--------------------+--------------------------+
|NU_ANO_CENSO|   NO_REGIAO|            NO_UF|sum(IN_AGUA_FILTRADA)|sum(IN_AGUA_POTAVEL)|sum(IN_AGUA_REDE_PUBLICA)|sum(IN_AGUA_POCO_ARTESIANO)|sum(IN_AGUA_CACIMBA)|sum(IN_AGUA_FONTE_RIO)|sum(IN_AGUA_INEXISTENTE)|sum(IN_ESGOTO_REDE_PUBLICA)|sum(IN_ESGOTO_FOSSA_SEPTICA)|sum(IN_ESGOTO_FOSSA_COMUM)|sum(IN_ESGOTO_FOSSA)|sum(IN_ESGOTO_INEXISTENTE)|
+------------+------------+-----------------+---------------------+--------------------+-------------------------+---------------------------+--------------------+----------------------+------------------------+---------------------------+----------------------------+--------------------------+-----------------

In [22]:
s3.save_s3_data(censoEscolarSaneamentoDF,'preprocessado','CENSO_ESCOLAR/DADOS_SANEAMENTO')
s3.save_s3_data(censoEscolarSaneamentoDF,'cliente','CENSO_ESCOLAR/DADOS_SANEAMENTO')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
s3.save_data_client_side(censoEscolarSaneamentoDF,'CENSO_ESCOLAR_DADOS_SANEAMENTO')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o2084.save.
: java.lang.ClassNotFoundException: Failed to find data source: com.microsoft.sqlserver.jdbc.spark. Please find packages at http://spark.apache.org/third-party-projects.html
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:679)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:733)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:967)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:304)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(Re

**Censo Escolar Energia**

In [61]:
censoEscolarEnergiaDF = censoEscolarDF.groupBy('NU_ANO_CENSO','NO_REGIAO','NO_UF').sum('IN_ENERGIA_REDE_PUBLICA','IN_ENERGIA_GERADOR','IN_ENERGIA_GERADOR_FOSSIL','IN_ENERGIA_OUTROS','IN_ENERGIA_RENOVAVEL','IN_ENERGIA_INEXISTENTE')
censoEscolarEnergiaDF.show()

+------------+---------+-------------------+----------------------------+-----------------------+------------------------------+----------------------+-------------------------+---------------------------+
|NU_ANO_CENSO|NO_REGIAO|              NO_UF|sum(IN_ENERGIA_REDE_PUBLICA)|sum(IN_ENERGIA_GERADOR)|sum(IN_ENERGIA_GERADOR_FOSSIL)|sum(IN_ENERGIA_OUTROS)|sum(IN_ENERGIA_RENOVAVEL)|sum(IN_ENERGIA_INEXISTENTE)|
+------------+---------+-------------------+----------------------------+-----------------------+------------------------------+----------------------+-------------------------+---------------------------+
|        2018|    Norte|               Par�|                        8144|                   1135|                             0|                   221|                        0|                       1407|
|        2018|    Norte|               Acre|                         905|                    105|                             0|                    20|                        0

In [20]:
s3.save_s3_data(censoEscolarEnergiaDF,'preprocessado','CENSO_ESCOLAR/DADOS_ENERGIA')
s3.save_s3_data(censoEscolarEnergiaDF,'cliente','CENSO_ESCOLAR/DADOS_ENERGIA')

22/06/22 02:25:28 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:25:28 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:25:28 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:25:36 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:25:36 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:25:43 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:25:43 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:25:49 WA

In [21]:
s3.save_data_client_side(censoEscolarEnergiaDF,'CENSO_ESCOLAR_DADOS_ENERGIA')

**Censo Escolar Reciclagem**

In [51]:
censoEscolarReciclagemDF = censoEscolarDF.groupBy('NU_ANO_CENSO','NO_REGIAO','NO_UF').sum('IN_LIXO_SERVICO_COLETA','IN_LIXO_QUEIMA','IN_LIXO_ENTERRA','IN_LIXO_DESTINO_FINAL_PUBLICO','IN_LIXO_DESCARTA_OUTRA_AREA','IN_LIXO_JOGA_OUTRA_AREA','IN_LIXO_OUTROS','IN_LIXO_RECICLA')
censoEscolarReciclagemDF.show()

+------------+---------+--------+--------------------+-----------------+---------------+----------------------+--------------+---------------+-----------------------------+---------------------------+-----------------------+--------------+---------------+
|NU_ANO_CENSO|NO_REGIAO|   NO_UF|        NO_MUNICIPIO|   NO_MESORREGIAO|NO_MICRORREGIAO|IN_LIXO_SERVICO_COLETA|IN_LIXO_QUEIMA|IN_LIXO_ENTERRA|IN_LIXO_DESTINO_FINAL_PUBLICO|IN_LIXO_DESCARTA_OUTRA_AREA|IN_LIXO_JOGA_OUTRA_AREA|IN_LIXO_OUTROS|IN_LIXO_RECICLA|
+------------+---------+--------+--------------------+-----------------+---------------+----------------------+--------------+---------------+-----------------------------+---------------------------+-----------------------+--------------+---------------+
|        2018|    Norte|Rond�nia|Alta Floresta D'O...|Leste Rondoniense|         Cacoal|                     0|             1|              1|                            0|                          0|                      0|        

In [23]:
s3.save_s3_data(censoEscolarReciclagemDF,'preprocessado','CENSO_ESCOLAR/DADOS_RECICLAGEM')
s3.save_s3_data(censoEscolarReciclagemDF,'cliente','CENSO_ESCOLAR/DADOS_RECICLAGEM')

22/06/22 02:32:50 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:32:51 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:32:51 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:32:57 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:32:57 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:33:02 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:33:03 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:33:08 WA

In [24]:
s3.save_data_client_side(censoEscolarReciclagemDF,'CENSO_ESCOLAR_DADOS_RECICLAGEM')

**Censo Escolar Predio**

In [52]:
censoEscolarPredioDF = censoEscolarDF.groupBy('NU_ANO_CENSO','NO_REGIAO','NO_UF').sum('IN_BIBLIOTECA','IN_BIBLIOTECA_SALA_LEITURA','IN_COZINHA','IN_DESPENSA','IN_DORMITORIO_ALUNO','IN_DORMITORIO_PROFESSOR','IN_LABORATORIO_CIENCIAS','IN_LABORATORIO_INFORMATICA','IN_PATIO_COBERTO','IN_PATIO_DESCOBERTO','IN_PARQUE_INFANTIL','IN_PISCINA','IN_QUADRA_ESPORTES','IN_QUADRA_ESPORTES_COBERTA','IN_QUADRA_ESPORTES_DESCOBERTA','IN_REFEITORIO','QT_SALAS_EXISTENTES','QT_SALAS_UTILIZADAS_DENTRO','QT_SALAS_UTILIZADAS_FORA','QT_SALAS_UTILIZADAS','QT_SALAS_UTILIZA_CLIMATIZADAS','QT_SALAS_UTILIZADAS_ACESSIVEIS','IN_ALIMENTACAO')
censoEscolarPredioDF.show()

+------------+---------+--------+--------------------+-----------------+---------------+-------------+--------------------------+----------+-----------+-------------------+-----------------------+-----------------------+--------------------------+----------------+-------------------+------------------+----------+------------------+--------------------------+-----------------------------+-------------+-------------------+--------------------------+------------------------+-------------------+-----------------------------+------------------------------+--------------+
|NU_ANO_CENSO|NO_REGIAO|   NO_UF|        NO_MUNICIPIO|   NO_MESORREGIAO|NO_MICRORREGIAO|IN_BIBLIOTECA|IN_BIBLIOTECA_SALA_LEITURA|IN_COZINHA|IN_DESPENSA|IN_DORMITORIO_ALUNO|IN_DORMITORIO_PROFESSOR|IN_LABORATORIO_CIENCIAS|IN_LABORATORIO_INFORMATICA|IN_PATIO_COBERTO|IN_PATIO_DESCOBERTO|IN_PARQUE_INFANTIL|IN_PISCINA|IN_QUADRA_ESPORTES|IN_QUADRA_ESPORTES_COBERTA|IN_QUADRA_ESPORTES_DESCOBERTA|IN_REFEITORIO|QT_SALAS_EXISTENTES|QT_

In [26]:
s3.save_s3_data(censoEscolarPredioDF,'preprocessado','CENSO_ESCOLAR/DADOS_PREDIO')
s3.save_s3_data(censoEscolarPredioDF,'cliente','CENSO_ESCOLAR/DADOS_PREDIO')

22/06/22 02:39:05 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:39:05 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:39:05 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:39:11 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:39:12 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:39:17 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:39:18 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:39:22 WA

In [27]:
s3.save_data_client_side(censoEscolarPredioDF,'CENSO_ESCOLAR_DADOS_PREDIO')

**Censo Escolar Informatica**

In [53]:
censoEscolarInformaticaDF = censoEscolarDF.groupBy('NU_ANO_CENSO','NO_REGIAO','NO_UF').sum('IN_DESKTOP_ALUNO','QT_DESKTOP_ALUNO','IN_COMP_PORTATIL_ALUNO','QT_COMP_PORTATIL_ALUNO','IN_TABLET_ALUNO','QT_TABLET_ALUNO','QT_COMPUTADOR','QT_COMP_ADMINISTRATIVO','IN_INTERNET','IN_INTERNET_ALUNOS','IN_INTERNET_ADMINISTRATIVO','IN_INTERNET_APRENDIZAGEM','IN_INTERNET_COMUNIDADE','IN_ACESSO_INTERNET_COMPUTADOR','IN_ACES_INTERNET_DISP_PESSOAIS','TP_REDE_LOCAL','IN_BANDA_LARGA')
censoEscolarInformaticaDF.show()

+------------+---------+--------+--------------------+-----------------+---------------+----------------+----------------+----------------------+----------------------+---------------+---------------+-------------+----------------------+-----------+------------------+--------------------------+------------------------+----------------------+-----------------------------+------------------------------+-------------+--------------+
|NU_ANO_CENSO|NO_REGIAO|   NO_UF|        NO_MUNICIPIO|   NO_MESORREGIAO|NO_MICRORREGIAO|IN_DESKTOP_ALUNO|QT_DESKTOP_ALUNO|IN_COMP_PORTATIL_ALUNO|QT_COMP_PORTATIL_ALUNO|IN_TABLET_ALUNO|QT_TABLET_ALUNO|QT_COMPUTADOR|QT_COMP_ADMINISTRATIVO|IN_INTERNET|IN_INTERNET_ALUNOS|IN_INTERNET_ADMINISTRATIVO|IN_INTERNET_APRENDIZAGEM|IN_INTERNET_COMUNIDADE|IN_ACESSO_INTERNET_COMPUTADOR|IN_ACES_INTERNET_DISP_PESSOAIS|TP_REDE_LOCAL|IN_BANDA_LARGA|
+------------+---------+--------+--------------------+-----------------+---------------+----------------+----------------+----------

In [29]:
s3.save_s3_data(censoEscolarInformaticaDF,'preprocessado','CENSO_ESCOLAR/DADOS_INFORMATICA')
s3.save_s3_data(censoEscolarInformaticaDF,'cliente','CENSO_ESCOLAR/DADOS_INFORMATICA')

22/06/22 02:47:04 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:47:05 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:47:05 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:47:11 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:47:11 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:47:16 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:47:16 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:47:22 WA

In [30]:
s3.save_data_client_side(censoEscolarInformaticaDF,'CENSO_ESCOLAR_DADOS_INFORMATICA')

**Censo Escolar Professores**

In [31]:
censoEscolarProfessoresDF = censoEscolarDF.groupBy('NU_ANO_CENSO','NO_REGIAO','NO_UF').sum('QT_FUNCIONARIOS','IN_PROF_ADMINISTRATIVOS','QT_PROF_ADMINISTRATIVOS','IN_PROF_SERVICOS_GERAIS','QT_PROF_SERVICOS_GERAIS','IN_PROF_BIBLIOTECARIO','QT_PROF_BIBLIOTECARIO','IN_PROF_SAUDE','QT_PROF_SAUDE','IN_PROF_COORDENADOR','QT_PROF_COORDENADOR','IN_PROF_FONAUDIOLOGO','QT_PROF_FONAUDIOLOGO','IN_PROF_NUTRICIONISTA','QT_PROF_NUTRICIONISTA','IN_PROF_PSICOLOGO','QT_PROF_PSICOLOGO','IN_PROF_ALIMENTACAO','QT_PROF_ALIMENTACAO','IN_PROF_PEDAGOGIA','QT_PROF_PEDAGOGIA','IN_PROF_SECRETARIO','QT_PROF_SECRETARIO','IN_PROF_SEGURANCA','QT_PROF_SEGURANCA','IN_PROF_MONITORES','QT_PROF_MONITORES','IN_PROF_GESTAO','QT_PROF_GESTAO','IN_PROF_ASSIST_SOCIAL','QT_PROF_ASSIST_SOCIAL')
censoEscolarProfessoresDF.show()

+------------+---------+--------+--------------------+-----------------+---------------+---------------+-----------------------+-----------------------+-----------------------+-----------------------+---------------------+---------------------+-------------+-------------+-------------------+-------------------+--------------------+--------------------+---------------------+---------------------+-----------------+-----------------+-------------------+-------------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+--------------+--------------+---------------------+---------------------+
|NU_ANO_CENSO|NO_REGIAO|   NO_UF|        NO_MUNICIPIO|   NO_MESORREGIAO|NO_MICRORREGIAO|QT_FUNCIONARIOS|IN_PROF_ADMINISTRATIVOS|QT_PROF_ADMINISTRATIVOS|IN_PROF_SERVICOS_GERAIS|QT_PROF_SERVICOS_GERAIS|IN_PROF_BIBLIOTECARIO|QT_PROF_BIBLIOTECARIO|IN_PROF_SAUDE|QT_PROF_SAUDE|IN_PROF_COORDENADOR|QT_PROF_COORD

In [32]:
s3.save_s3_data(censoEscolarProfessoresDF,'preprocessado','CENSO_ESCOLAR/DADOS_PROFESSORES')
s3.save_s3_data(censoEscolarProfessoresDF,'cliente','CENSO_ESCOLAR/DADOS_PROFESSORES')

22/06/22 02:55:52 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:55:52 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:55:52 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:55:59 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:55:59 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:56:06 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:56:06 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 02:56:13 WA

In [33]:
s3.save_data_client_side(censoEscolarProfessoresDF,'CENSO_ESCOLAR_DADOS_PROFESSORES')

**Censo Escolar Ensino**

In [34]:
censoEscolarEnsinoDF = censoEscolarDF.groupBy('NU_ANO_CENSO','NO_REGIAO','NO_UF').sum('IN_EXAME_SELECAO','IN_DIURNO','IN_NOTURNO','IN_EAD','IN_BAS','IN_INF','IN_INF_CRE','IN_INF_PRE','IN_FUND','IN_FUND_AI','IN_FUND_AF','IN_MED','IN_PROF','IN_PROF_TEC','IN_EJA','IN_EJA_FUND','IN_EJA_MED','IN_ESP','IN_ESP_CC','IN_ESP_CE')
censoEscolarEnsinoDF.show()

+------------+---------+--------+--------------------+-----------------+---------------+----------------+---------+----------+------+------+------+----------+----------+-------+----------+----------+------+-------+-----------+------+-----------+----------+------+---------+---------+
|NU_ANO_CENSO|NO_REGIAO|   NO_UF|        NO_MUNICIPIO|   NO_MESORREGIAO|NO_MICRORREGIAO|IN_EXAME_SELECAO|IN_DIURNO|IN_NOTURNO|IN_EAD|IN_BAS|IN_INF|IN_INF_CRE|IN_INF_PRE|IN_FUND|IN_FUND_AI|IN_FUND_AF|IN_MED|IN_PROF|IN_PROF_TEC|IN_EJA|IN_EJA_FUND|IN_EJA_MED|IN_ESP|IN_ESP_CC|IN_ESP_CE|
+------------+---------+--------+--------------------+-----------------+---------------+----------------+---------+----------+------+------+------+----------+----------+-------+----------+----------+------+-------+-----------+------+-----------+----------+------+---------+---------+
|        2018|    Norte|Rond�nia|Alta Floresta D'O...|Leste Rondoniense|         Cacoal|              NA|        1|         0|     0|     1|     0| 

In [35]:
s3.save_s3_data(censoEscolarEnsinoDF,'preprocessado','CENSO_ESCOLAR/DADOS_ENSINO')
s3.save_s3_data(censoEscolarEnsinoDF,'cliente','CENSO_ESCOLAR/DADOS_ENSINO')

22/06/22 03:05:00 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 03:05:01 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 03:05:01 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 03:05:08 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 03:05:09 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 03:05:14 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 03:05:14 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 03:05:21 WA

In [36]:
s3.save_data_client_side(censoEscolarEnsinoDF,'CENSO_ESCOLAR_DADOS_ENSINO')

**Censo Escolar Matriculas**

In [37]:
censoEscolarMatriculasDF = censoEscolarDF.groupBy('NU_ANO_CENSO','NO_REGIAO','NO_UF').sum('QT_MAT_BAS','QT_MAT_INF','QT_MAT_INF_CRE','QT_MAT_INF_PRE','QT_MAT_FUND','QT_MAT_FUND_AI','QT_MAT_FUND_AF','QT_MAT_MED','QT_MAT_PROF','QT_MAT_PROF_TEC','QT_MAT_EJA','QT_MAT_EJA_FUND','QT_MAT_EJA_MED','QT_MAT_ESP','QT_MAT_ESP_CC','QT_MAT_ESP_CE','QT_MAT_BAS_FEM','QT_MAT_BAS_MASC','QT_MAT_BAS_ND','QT_MAT_BAS_BRANCA','QT_MAT_BAS_PRETA','QT_MAT_BAS_PARDA','QT_MAT_BAS_AMARELA','QT_MAT_BAS_INDIGENA','QT_MAT_BAS_0_3','QT_MAT_BAS_4_5','QT_MAT_BAS_6_10','QT_MAT_BAS_11_14','QT_MAT_BAS_15_17','QT_MAT_BAS_18_MAIS','QT_MAT_BAS_D','QT_MAT_BAS_N','QT_MAT_BAS_EAD','QT_MAT_INF_INT','QT_MAT_INF_CRE_INT','QT_MAT_INF_PRE_INT','QT_MAT_FUND_INT','QT_MAT_FUND_AI_INT','QT_MAT_FUND_AF_INT','QT_MAT_MED_INT')
censoEscolarMatriculasDF.show()

+------------+---------+--------+--------------------+-----------------+---------------+----------+----------+--------------+--------------+-----------+--------------+--------------+----------+-----------+---------------+----------+---------------+--------------+----------+-------------+-------------+--------------+---------------+-------------+-----------------+----------------+----------------+------------------+-------------------+--------------+--------------+---------------+----------------+----------------+------------------+------------+------------+--------------+--------------+------------------+------------------+---------------+------------------+------------------+--------------+
|NU_ANO_CENSO|NO_REGIAO|   NO_UF|        NO_MUNICIPIO|   NO_MESORREGIAO|NO_MICRORREGIAO|QT_MAT_BAS|QT_MAT_INF|QT_MAT_INF_CRE|QT_MAT_INF_PRE|QT_MAT_FUND|QT_MAT_FUND_AI|QT_MAT_FUND_AF|QT_MAT_MED|QT_MAT_PROF|QT_MAT_PROF_TEC|QT_MAT_EJA|QT_MAT_EJA_FUND|QT_MAT_EJA_MED|QT_MAT_ESP|QT_MAT_ESP_CC|QT_MAT_ESP_

In [38]:
s3.save_s3_data(censoEscolarMatriculasDF,'preprocessado','CENSO_ESCOLAR/DADOS_MATRICULAS')
s3.save_s3_data(censoEscolarMatriculasDF,'cliente','CENSO_ESCOLAR/DADOS_MATRICULAS')

22/06/22 03:12:43 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 03:12:43 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 03:12:43 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 03:12:51 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 03:12:51 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 03:12:58 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 03:12:59 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/22 03:13:07 WA

In [39]:
s3.save_data_client_side(censoEscolarMatriculasDF,'CENSO_ESCOLAR_DADOS_MATRICULAS')

In [32]:
s3.save_data_client_side(censoEscolarDF,'CENSO_ESCOLAR')

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 2) / 8]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/site-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib64/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 